In [8]:
#1. IMPORTS
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from tqdm import tqdm
import time
from PIL import Image 

#2. Use Faster Image Preprocessing (128×128)
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

#3. Load Dataset (Faster DataLoader)

import os
import shutil
import random

source_dir = "garbage_classification"       # your only folder
dest_dir = "dataset"         # new split folder
train_ratio = 0.8            # 80% training, 20% testing

# Create output folders
train_dir = os.path.join(dest_dir, "train")
test_dir  = os.path.join(dest_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir,  exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)

    if not os.path.isdir(class_path):
        continue
    
    images = os.listdir(class_path)
    random.shuffle(images)

    split_idx = int(len(images) * train_ratio)
    train_files = images[:split_idx]
    test_files  = images[split_idx:]

    # Create class folders inside train/test
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name),  exist_ok=True)

    # Move files to train folder
    for img in train_files:
        shutil.copy(
            os.path.join(class_path, img),
            os.path.join(train_dir, class_name, img)
        )

    # Move files to test folder
    for img in test_files:
        shutil.copy(
            os.path.join(class_path, img),
            os.path.join(test_dir, class_name, img)
        )

    print(f"Class '{class_name}' → {len(train_files)} train, {len(test_files)} test")

print("Dataset split completed successfully!")
train_data = datasets.ImageFolder("dataset/train", transform=train_transforms)
test_data  = datasets.ImageFolder("dataset/test", transform=test_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

class_names = train_data.classes

#4. FAST Model (MobileNetV2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

class SmallCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Linear(64 * 16 * 16, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = SmallCNN(num_classes=len(class_names)).to(device)

# Replace classification layer

model1 = model1.to(device)

#5. Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
epochs = 10

#6. SUPER-FAST TRAIN LOOP (with progress bar)
for epoch in range(epochs):
    print(f"\n🔥 Epoch {epoch+1}/{epochs}")
    model1.train()

    start = time.time()
    total_correct = 0
    total_samples = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model1(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    train_acc = total_correct / total_samples
    print(f"✔ Train Accuracy: {train_acc:.4f}")
    print(f"⏱️ Epoch Time: {time.time() - start:.2f} seconds")

#8. CONFUSION MATRIX + VISUALIZATION
from sklearn.metrics import confusion_matrix
y_true = []
y_pred = []

model1.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model1(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

cm = confusion_matrix(y_true, y_pred)

def predict_image(image_path, model):
    model.eval()

    img = Image.open(image_path).convert("RGB")
    transform = test_transforms

    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)

    class_name = class_names[predicted.item()]
    return class_name, confidence.item()

Class 'battery' → 756 train, 189 test
Class 'biological' → 788 train, 197 test
Class 'brown-glass' → 485 train, 122 test
Class 'cardboard' → 712 train, 179 test
Class 'clothes' → 4260 train, 1065 test
Class 'green-glass' → 503 train, 126 test
Class 'metal' → 615 train, 154 test
Class 'paper' → 840 train, 210 test
Class 'plastic' → 692 train, 173 test
Class 'shoes' → 1581 train, 396 test
Class 'trash' → 557 train, 140 test
Class 'white-glass' → 620 train, 155 test
Dataset split completed successfully!
Running on: cpu

🔥 Epoch 1/10


100%|██████████| 485/485 [05:18<00:00,  1.52it/s]


✔ Train Accuracy: 0.5655
⏱️ Epoch Time: 318.29 seconds

🔥 Epoch 2/10


100%|██████████| 485/485 [05:27<00:00,  1.48it/s]


✔ Train Accuracy: 0.6830
⏱️ Epoch Time: 327.84 seconds

🔥 Epoch 3/10


...

✔ Train Accuracy: 0.8199
⏱️ Epoch Time: 315.32 seconds

🔥 Epoch 9/10


100%|██████████| 485/485 [05:43<00:00,  1.41it/s]


✔ Train Accuracy: 0.8355
⏱️ Epoch Time: 343.37 seconds

🔥 Epoch 10/10


100%|██████████| 485/485 [05:42<00:00,  1.41it/s]


✔ Train Accuracy: 0.8483
⏱️ Epoch Time: 342.91 seconds


In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt

def plot_training(history):
    train_losses = history['train_loss']
    val_losses = history['val_loss']
    train_acc = history['train_acc']
    val_acc = history['val_acc']

    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(14,5))

    # ---- Loss Plot ----
    plt.subplot(1,2,1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Val Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()

    # ---- Accuracy Plot ----
    plt.subplot(1,2,2)
    plt.plot(epochs, train_acc, label='Train Accuracy')
    plt.plot(epochs, val_acc, label='Val Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()

    plt.show()

In [ ]:
%write new.

predicted_class, confidence = predict_image("test.jpg", model)
print(f"Predicted class: {predicted_class}, Confidence: {confidence:.4f}")

Predicted class: white-glass, Confidence: 0.6022
